In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# Read the data
X = pd.read_csv('./train.csv', index_col='Id')
X_test = pd.read_csv('./test.csv', index_col='Id')


# Remove rows with missing target (y)
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# Drop columns with missing data
cols_with_missing_data = [col for col in X.columns if X[col].isnull().any()]
X.drop(cols_with_missing_data, axis=1, inplace=True)
X_test.drop(cols_with_missing_data, axis=1, inplace=True)

# Create our train/test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [34]:
X_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,108,0,0,260,0,0,7,2007,New,Partial
871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,8,2009,WD,Normal
93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,...,0,44,0,0,0,0,8,2009,WD,Normal
818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,59,0,0,0,0,0,7,2008,WD,Normal
303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,...,81,0,0,0,0,0,1,2006,WD,Normal


In [35]:
# Build our scoring function for our different approaches to handling categorical data

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [36]:
# Approach 1, drop categorical data

drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

# Calculate MAE for this approach
print('MAE for dropping categorical data:')
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE for dropping categorical data:
17837.82570776256


In [37]:
# Before moving into our next approach, let's investigate our data. We can print the unique values within
# a column. Specifically, we'll print values of column Condition2

print("\nUnique values in 'Condition2' column in training data:", X_train['Condition2'].unique())
print("\nUnique values in 'Condition2' column in validation data:", X_valid['Condition2'].unique())


Unique values in 'Condition2' column in training data: ['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']

Unique values in 'Condition2' column in validation data: ['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']


In [38]:
# Approach 2, ordinal encoding
#
# The easiest approach to categorical values that appear in training data but not validation data
# (or vice versa) is to drop those columns. Here we figure out which columns those are

object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

good_label_cols = [col for col in object_cols if 
             set(X_valid[col]).issubset(set(X_train[col]))]

bad_cols = list(set(object_cols)-set(good_label_cols))

# Drop columns that will not be encoded
label_X_train = X_train.drop(bad_cols, axis=1)
label_X_valid = X_valid.drop(bad_cols, axis=1)

ordinal_encoder = OrdinalEncoder()
label_X_train[good_label_cols] = ordinal_encoder.fit_transform(label_X_train[good_label_cols])
label_X_valid[good_label_cols] = ordinal_encoder.transform(label_X_valid[good_label_cols])

# Calculate MAE of Approach 2 (ordinal encoding)

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
17098.01649543379


In [39]:
# Approach 3, one-hot encoding

# Get the number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print the number of unique entries by column in ascending order
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

In [40]:
# As shown above there are a few columns with more than 10 unique entries. If we used cardinality to encode these
# our dataset could grow rather large

# Let's instead one-hot encode low cardinal (10 or fewer) categorical variables
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Collect high cardinality variables (>= 10)
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

Categorical columns that will be one-hot encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Exterior2nd', 'Neighborhood', 'Exterior1st']


In [41]:
# Getting back to one-hot encoding (Approach 3)

# First drop the high_cardinality_cols from our datasets

X_train_copy = X_train.copy()
X_valid_copy = X_valid.copy()

# Start one-hot encoding
hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_train_cols = pd.DataFrame(hot_encoder.fit_transform(X_train_copy[low_cardinality_cols]))
OH_valid_cols = pd.DataFrame(hot_encoder.transform(X_valid_copy[low_cardinality_cols]))

# Put the indexes back
OH_train_cols.index = X_train_copy.index
OH_valid_cols.index = X_valid_copy.index

# Remove high cardinality columns
highs_removed_train_cols = X_train.drop(object_cols, axis=1)
highs_removed_valid_cols = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded cols to our data
OH_X_train = pd.concat([highs_removed_train_cols, OH_train_cols], axis=1)
OH_X_valid = pd.concat([highs_removed_valid_cols, OH_valid_cols], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
17525.345719178084
